In [ ]:
    %run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/barCharts.ipynb
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

Selectors :
   * percentageAlternativeResPlayer : checkboxe (T/F) value to display the bar chart in percentage or not
   * xaxisAlternativeResPlayer : dropdown value indicating the number of bar per chart (Alls, 5, 25, 50, 10 first, 10 last)
   * orderAlternativeResPlayer : dropdown value indicating how to order xlabel : alphabetically or by y value (hight to low or low to hight)
   * orderLabelValuesAlternativeResPlayer : dropdown value indicating how to order labels : by the sum of the 2 superposed values or by true or false response

In [ ]:
#constructing a dropdown to select how many data we want to see in visualisations for x axis
xaxisAlternativeResPlayer=widgets.Dropdown(options=[('All data',0),('5 per visualisation',5),('25 per visualisation',25),('50 per visualisation',50),('10 first',-1),('10 last',-2)],
                            description='x axe',
                            disabled=False)
#constructing a dropdown to order for alphabetic keys or for values
orderAlternativeResPlayer=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
                            description='Order',
                            disabled=False)
#constructing a checkboxe to view chart per percentage per player
percentageAlternativeResPlayer=widgets.Checkbox(value=False,
                                        description='Show % per player')
#constructing a dropdown value indicating how to order labels : by the sum of the 2 superposed values or by true or false response
orderLabelValuesAlternativeResPlayer=widgets.Dropdown(options=[("Order by all true and false responses values",0),
                                                ("Order by true responses values","true"),
                                                ("Order by false responses values","false")],
                                        description='Change order label',
                                        disabled=False)


***vis_correct_incorrect_player* function**:
* Output : the widgets selector and the outputAltPerPlayer in a list to be display in a VBox
* outputAltPerPlayer : a bar chart showing for each user the number of correct and incorrect alternatives selected in multiple-choice questions

In [1]:
def vis_correct_incorrect_player():
    def display_correct_incorrect_player(change):
        if orderAlternativeResPlayer.value==0 or percentageAlternativeResPlayer.value:
            orderLabelValuesAlternativeResPlayer.layout=layout_hidden
        else:
            orderLabelValuesAlternativeResPlayer.layout=layout_visible
        with outputAltPerPlayer:
            clear_output(wait=False)
            construct_correct_incorrect_player()
            clear_output(wait=True)
    xaxisAlternativeResPlayer.observe(display_correct_incorrect_player, 'value')
    orderAlternativeResPlayer.observe(display_correct_incorrect_player, 'value')
    percentageAlternativeResPlayer.observe(display_correct_incorrect_player, 'value')
    orderLabelValuesAlternativeResPlayer.observe(display_correct_incorrect_player, 'value')
    display_correct_incorrect_player(None)
    return [percentageAlternativeResPlayer,
            xaxisAlternativeResPlayer,
            orderAlternativeResPlayer,
            orderLabelValuesAlternativeResPlayer,
            outputAltPerPlayerUpdate,
            outputAltPerPlayer]


***construct_correct_incorrect_player* function**:

Output :  Displays bar chart showing for each user the number of correct and incorrect alternatives selected in multiple-choice questions

In [ ]:
def construct_correct_incorrect_player():
    #get all players and alternatives selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    alternatives_list_selected=checkboxesAlternativeSelected.get_selected_options()
    if len(alternatives_list_selected)>0 and len(playersSelected)>0:
        if xaxisAlternativeResPlayer.value in [0,-1,-2]:
            nbgraph=1
        else:
            div=len(playersSelected)/xaxisAlternativeResPlayer.value
            nbgraph=math.ceil(div)
        accordionResPerPlayer=widgets.Accordion(children=[widgets.Output() for i in range(nbgraph)],
                                         selected_index=None)
        if nbgraph>1:
            for i in range(nbgraph):
                accordionResPerPlayer.set_title(i, str(i+1)+"/"+str(nbgraph))
        display(accordionResPerPlayer)
        def on_selected_index_ResPerPlayer_change(change):
            n=change['new']
            if n!=None:
                #close and clear previous fig of alternatives
                for fig in figAltPlayer:
                    plt.close(fig=fig)
                figAltPlayer.clear()
                #get all players and alternatives selected
                playersSelected=checkboxesPlayersSelected.get_selected_options()
                alternatives_list_selected=checkboxesAlternativeSelected.get_selected_options()
                true_false_per_player = {}
                total_max=0
                for player in playersSelected:
                    true_false_per_player[player]={}
                    if len(players_info[player]["alternatives"].keys())>0:
                        total_true = 0
                        total_false = 0
                        for alt in alternatives_list_selected:
                            if alt in players_info[player]["alternatives"].keys():
                                _, results = zip(*players_info[player]["alternatives"][alt]) # take only T/F result values
                                total_true += results.count("True") + results.count(True)
                                total_false += results.count("False") + results.count(False)
                                if total_true+total_false>total_max:
                                    total_max=total_true+total_false
                        true_false_per_player[player]["true"]=total_true
                        true_false_per_player[player]["false"]=total_false
                    else:
                        true_false_per_player[player]["true"]=0
                        true_false_per_player[player]["false"]=0
                total_max+=0.5
                #define order of labels values
                if orderLabelValuesAlternativeResPlayer.value==0:
                    lambdafunc=lambda item:item[1]["true"]+item[1]["false"]
                else:
                    lambdafunc=lambda item:item[1][orderLabelValuesAlternativeResPlayer.value]
                if percentageAlternativeResPlayer.value:
                    lambdafunc=lambda item:item[1]["true"]/(item[1]["true"]+item[1]["false"])
                #order alls values
                true_false_per_player = orderDict(true_false_per_player,orderAlternativeResPlayer.value,lambdafunc)
                #separe in sub dict in  function of xaxis dropout selected
                nbgraphshown, true_false_per_player,N = organizeDictPerxAxisSelected(playersSelected,xaxisAlternativeResPlayer.value, true_false_per_player)
                with accordionResPerPlayer.children[n]:
                    if len(alternatives_list_selected)>0 and len(playersSelected)>0:
                        #get value for true and false answer for this graph
                        true_per_player=[true_false_per_player[n][player]["true"] for player in true_false_per_player[n].keys()]
                        false_per_player=[true_false_per_player[n][player]["false"] for player in true_false_per_player[n].keys()]
                        list_player=[player for player in true_false_per_player[n].keys()]
                        if n<nbgraphshown-1:
                            ind = np.arange(N)
                        else:
                            ind = np.arange(len(list_player))
                    #create new fig with custom name in the tab
                    name=filename+ "_correct_incorrect_per_player"
                    if nbgraphshown>1:
                        name+="_"+str(n+1)+"_"+str(nbgraphshown)
                    fig = plt.figure(num=name, constrained_layout=True)
                    fig.canvas.header_visible = False
                    fig.canvas.layout.min_height = '400px'
                    figAltPlayer.append(fig)
                    #display graph
                    if percentageAlternativeResPlayer.value:
                        # Bar chart showing for each user the percentage of incorrect alternatives
                        # selected in multiple-choice questions
                        get_percentage_bar(true_per_player,false_per_player, list_player)
                        ylabel="percentage of good responses"
                        title="Percentage of good responses per player"
                    else:
                        # Bar chart showing for each user the number of correct and incorrect alternatives
                        # selected in multiple-choice questions
                        get_two_superposed_values_bars(ind,list_player,
                                                       true_per_player, "Correct",'g',
                                                       false_per_player, "Incorrect",'r')
                        ylabel="Number of responses"
                        title="Correct and incorrect responses per player"
                    #add axes labels and title
                    plt.ylabel(ylabel)
                    plt.xlabel("Player")
                    #orient xticks
                    plt.xticks(rotation=90)
                    plt.title(title)
                    plt.show()
        accordionResPerPlayer.observe(on_selected_index_ResPerPlayer_change, 'selected_index')
        accordionResPerPlayer.selected_index=0
    else:
        #create new fig with custom name in the tab
        name=filename+ "_correct_incorrect_per_player"
        fig = plt.figure(num=name, constrained_layout=True)
        fig.canvas.header_visible = False
        fig.canvas.layout.min_height = '400px'
        figAltQuestion.append(fig)
        #display no data to fill vis
        noDataToFillVis(10)
        if percentageAlternativeResPlayer.value:
            ylabel="Percentage of good responses"
            title="Percentage of good responses per player"
        else:
            ylabel="Number of responses"
            title="Number of correct and incorrect responses per player"
        #add axes labels and title
        plt.xlabel("Player")
        plt.ylabel(ylabel)
        plt.title(title)
        plt.show()
    with outputAltPerPlayerUpdate:
        clear_output(wait=False)